Demonstrate measuring the nominal delay between the input and output DDS blocks.

In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange
import Scan
import datetime
soc = TopSoc()

In [ ]:
fMixer = soc.fsIn/2/2
soc.set_mixer(fMixer)

In [ ]:
scan = Scan.Scan(soc)
outCh = 123
nf = 5
nominalDelay = scan.measureNominalDelay(outCh, decimation=2, nf=nf, doPlot=True)


In [ ]:
Scan.fscanPlot(scan.mndScan,0)

In [ ]:
Scan.applyDelay(scan.mndScan, nominalDelay)

In [ ]:
Scan.fscanPlot(scan.mndScan,0)

In [ ]:
plt.plot(1000*scan.dfs,scan.fis, "-x", label='fis')
plt.plot(1000*scan.dfs,scan.ufis, "-+", label='ufis')
plt.legend()
plt.xlabel("delta frequency (kHz)")
plt.ylabel("fi (Radians)")

In [ ]:
nToDo = 100
delays = np.zeros(nToDo)
for i in trange(nToDo):
    delays[i] = scan.measureNominalDelay(outCh, decimation=2, nf=5, doPlot=i==0)

In [ ]:
plt.hist(delays)
resolution = 1000*delays.std()/(2*np.pi)
_ = plt.title("mean=%.2f   sigma=%.4f resolution=%.2f nSec"%(delays.mean(),delays.std(), resolution))

In [ ]:
length = 4+34+34 # in nsec
nominalDelay = scan.measureNominalDelay(outCh, decimation=2, nf=nf, doPlot=False)
with open("junk.txt","a") as file:
    file.write("%s %d %f\n"%(datetime.datetime.now(),length,nominalDelay))
with open("junk.txt","r") as file:
    lines = file.readlines()

lengths = np.zeros(len(lines))
delays = np.zeros(len(lines))
for i,line in enumerate(lines):
    lengths[i] = line.split()[2]
    delays[i] = line.split()[3] 
plt.plot(lengths, delays, 'o')
plt.plot(lengths[-1], delays[-1], 'ro')

In [ ]:
ulengths = np.sort(np.unique(lengths))
print(ulengths)
udelays = np.zeros(len(ulengths))
for i,ulength in enumerate(ulengths):
    udelays[i] = delays[lengths==ulength].mean()
#plt.plot(ulengths, udelays, "o")
dlengths = ulengths - ulengths[0]
ddelays = udelays - udelays[0]
#plt.plot(dlengths, ddelays, 'o')
dusecs = 1000*ddelays/(2*np.pi)
plt.plot(dlengths, dusecs, "o")
plt.plot([0, ulengths[-1]],[0, -ulengths[-1]], 'r')
plt.xlabel("cable length - minimum cable length (nSec)")
plt.ylabel("calculated delay (nSec)")